In [ ]:
import conda_scripts
import pandas as pd
import geopandas as gpd
import datetime
import os

In [ ]:
wl = conda_scripts.wiski.wiski.get_gw_stations_wellmeta_in_basin(basins = ['Son*'])

In [ ]:
stats = wl.dropna(subset='station_no')
stats = stats.dropna(subset='station_longitude')
stats = stats.loc[stats.station_longitude<0]

In [ ]:
stats.loc[:, 'station_latitude'] = pd.to_numeric(stats.station_latitude, errors='coerce')
stats.loc[:, 'station_longitude'] = pd.to_numeric(stats.station_longitude, errors='coerce')
stats = stats.astype({'station_latitude': float, 'station_longitude': float}, errors='ignore')

stats = gpd.GeoDataFrame(stats, geometry=gpd.points_from_xy(stats.station_longitude, stats.station_latitude),
                         crs=4326).to_crs(2226)

In [ ]:
stats.sort_values('station_longitude')

In [ ]:
wshed = conda_scripts.load_datasets.load_watersheds.load('SON')

In [ ]:
stats = gpd.clip(stats, wshed)

In [ ]:
stats

In [ ]:
k = conda_scripts.wiski.wiski.get_kiwis()

In [ ]:
# load parameters for basin

locs = ['ADR*', 'PLP*', 'PRM*', 'Son*', 'UPC*' ,'UPE*' ,'WRM*']

son = [k.get_timeseries_list(station_name=bas, ts_name='01.Fin*,DayMeanEdit*,02.Final*',
                             parametertype_name='Groundwater_elevation',
                             return_fields=['station_name', 'station_no', 'station_id', 'ts_id', 'ts_name',
                                            'parametertype_id', 'parametertype_name', 'coverage']

                             )

       for bas in locs]

In [ ]:
pars = pd.concat(son)
pars = pars.loc[pd.to_datetime(pars.loc[:,'from']).notnull()]

In [ ]:
pars = pars.loc[pars.station_no.isin(stats.station_no)]

In [ ]:
df = []
for name, group in pars.groupby('ts_id'):
    ts = k.get_timeseries_values(ts_id=name,
                                 **{'from': datetime.datetime(1969, 12, 1)},
                                 return_fields=['Value', 'Timestamp', 'Quality Code'])
    if ts.shape[0] > 0:
        ts.loc[:, 'ts_id'] = name

        df.extend([ts])

tsall = pd.concat(df)

In [ ]:
tsall.index = conda_scripts.wiski.wiski.tz_fix(tsall)

In [ ]:
tsall.shape

In [ ]:
tsall.to_csv(os.path.join('waterlevel','wl.csv'))
pars.to_csv(os.path.join('waterlevel','pars.csv'))
stats.to_csv(os.path.join('waterlevel','stats.csv'))